In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
import copy

## Парсинг amc

In [2]:
def read_frame(frame, frames_dict):
    # parse joints
    for joint in frame:
        joint_info = joint.strip().split()
        joint_name = joint_info[0]
        joint_info.remove(joint_name)
        
        for i in range(len(joint_info)):
            coord = joint_info[i]
            frames_dict[joint_name + '_' + str(i)] = float(coord)
            
    return frames_dict

In [3]:
def parse_amc(filepath):
    with open(filepath) as f:
        content = f.read().splitlines()

    for idx, line in enumerate(content):
        if line == ':DEGREES':
            content = content[idx+1:]
            break
            
    # get idx of frames beginning
    frame_beginning = []
    for idx, line in enumerate(content):
        splitted_line = line.strip().split()
        if len(splitted_line) == 1:
            frame_beginning.append(idx)
    # parse frames
    all_frames = []
    for i in range(len(frame_beginning)-1):
        frame = content[frame_beginning[i]:frame_beginning[i+1]]
        frames_dict = {}
        frames_dict['frame_number'] = frame[0]
        # parse joints
        frames_dict = read_frame(frame[1:], frames_dict)
        all_frames.append(frames_dict)

    all_data = pd.DataFrame(all_frames)            
        
    return all_data

In [4]:
content = parse_amc('../data/02_01.amc')

In [5]:
content.head()

,frame_number,head_0,head_1,head_2,lclavicle_0,lclavicle_1,lfemur_0,lfemur_1,lfemur_2,lfingers_0,...,rwrist_0,thorax_0,thorax_1,thorax_2,upperback_0,upperback_1,upperback_2,upperneck_0,upperneck_1,upperneck_2
0,1,2.97226,2.54741,-1.92752,-1.620100e-14,-1.431250e-14,-28.0191,-1.020780,-20.1783,7.12502,...,-16.7798,-0.599593,1.31944,-2.05527,3.47255,2.55226,1.65684,1.617840,5.61246,-6.40733
1,2,3.01316,2.59206,-1.92353,-3.061280e-14,-7.951390e-16,-27.7269,-0.953135,-20.5822,7.12502,...,-16.5037,0.078456,1.35992,-1.86912,3.91254,2.58553,1.67171,1.419540,5.71695,-6.43796
2,3,2.90215,2.58200,-1.86960,3.975690e-15,-1.033680e-14,-27.6281,-0.810148,-21.0041,7.12502,...,-16.2262,0.211031,1.41449,-1.83734,3.96845,2.68906,1.61541,1.081860,5.66983,-6.25219
3,4,2.73437,2.55894,-1.86943,1.376580e-14,0.000000e+00,-27.6637,-0.391122,-21.6105,7.12502,...,-16.0827,0.127573,1.45150,-1.90802,3.81888,2.77415,1.44970,0.706102,5.58519,-6.09066
4,5,2.59153,2.57436,-1.87971,-2.042510e-14,-7.156250e-15,-27.6786,-0.161279,-21.8039,7.12502,...,-16.0025,0.158931,1.46518,-1.93228,3.70601,2.80336,1.40701,0.388960,5.58739,-5.98980


In [6]:
content.shape

(342, 63)

## Парсинг asf

In [106]:
def is_float(s):
    
    try:
        float(s)
        return float(s)
    
    except ValueError:
        return False
    
def is_number(s):
    
    if s.isdigit() == False:
        float_s = is_float(s)
        if float_s == False:
            val = False
        else:
            val = float_s
    else:
        val = int(s)
    return val
        
 
    

def read_bone_params(bone_data):
    
    clear_bone_data = [line for line in bone_data if ('end' in line)|('id' in line)|('begin' in line) == False] 
    
    # merge in one line all "limits"
    for id, param in enumerate(clear_bone_data):
        if param.strip().split()[0] == 'limits':
            clear_bone_data_new = clear_bone_data[:id]
            new_limits = "".join(clear_bone_data[id:])
            clear_bone_data_new.append(new_limits)
            break
        else:
            clear_bone_data_new = clear_bone_data  
    
    params_dict = {}
    for line in clear_bone_data_new:
        str_data = line.strip().split()
        if str_data[0] == 'name':
            bone_name = str_data[1]   
        else:
            values = list(str_data[1:])
            values_new = []
            for s in values:
                par = is_number(s)                             
                if  type(par) != bool:
                    val = par
                else:
                    val = s
                values_new.append(val)    
            params_dict[str_data[0]] = values_new
            
    bone_params = {}
    bone_params[bone_name] = params_dict   
    
    return bone_params


def parse_asf(filepath):

    with open(filepath) as f:
        content = f.read().splitlines()

    #считываем нижний блок с расписанной архитектурой
    hierarchy_beginning = []
    for idx, line in enumerate(content):
        splitted_line = line.strip().split()
        if line.rfind('hierarchy') > -1:
            hierarchy_start=idx
        if line.rfind('bonedata') > -1:
            bonedata_start=idx

    # надо теперь посчитать сколько bones
    bone_types = []
    bone_types_id = []
    for idx, line in enumerate(content):
        if (line.rfind('name') > -1 and idx > bonedata_start):
            bone_types_id.append(idx)
            # надо теперь посчитать сколько bones
            splitted_line = line.strip().split()
            #но и сохраняем
            bone_types.append(splitted_line[1])
            
    bone_types_iter = copy.deepcopy(bone_types_id)
    bone_types_iter.append(hierarchy_start)
    all_params = {}

    for i in range(len(bone_types_iter)-1):
        bone_data = content[bone_types_iter[i]:bone_types_iter[i+1]]
        bone_params = read_bone_params(bone_data)
        all_params = dict(all_params, **bone_params)
        
    return all_params 

In [104]:
filepath = '../data/02.asf'
all_p = parse_asf(filepath)

In [105]:
all_p

{'head': {'axis': [0, 0, 0, 'XYZ'],
  'direction': [0.008024, 0.999444, -0.0323691],
  'dof': ['rx', 'ry', 'rz'],
  'length': [1.6265],
  'limits': ['(-20.0', '45.0)', '(-30.0', '30.0)', '(-30.0', '30.0)']},
 'lclavicle': {'axis': [0, 0, 0, 'XYZ'],
  'direction': [0.967826, 0.247107, -0.0474446],
  'dof': ['ry', 'rz'],
  'length': [3.6598],
  'limits': ['(-20.0', '10.0)', '(0.0', '20.0)']},
 'lfemur': {'axis': [0, 0, 20, 'XYZ'],
  'direction': [0.34202, -0.939693, 0],
  'dof': ['rx', 'ry', 'rz'],
  'length': [7.59371],
  'limits': ['(-160.0', '20.0)', '(-70.0', '70.0)', '(-60.0', '70.0)']},
 'lfingers': {'axis': [-8.53362e-14, 90, 90, 'XYZ'],
  'direction': [1, -4.48731e-11, 1.33738e-25],
  'dof': ['rx'],
  'length': [0.533057],
  'limits': ['(0.0', '90.0)']},
 'lfoot': {'axis': [-90.0, 7.62852e-16, 20, 'XYZ'],
  'direction': [0.0886837, -0.243657, 0.965798],
  'dof': ['rx', 'rz'],
  'length': [2.2218],
  'limits': ['(-45.0', '90.0)', '(-70.0', '20.0)']},
 'lhand': {'axis': [-4.26681e-